# COM6012 - 2017: Courserwork 2

Deadline: 11:59PM on Thursday 9 March 2017

Submission: via SageMatchCloud (We will collect from you automatically.)

## Exercise 1 [8 marks]

In this exercise, you will study parallelization in detail by analyzing a text file and examining the time cost.

Please do the following:

Step 1. Download [Enron data](https://archive.ics.uci.edu/ml/machine-learning-databases/bag-of-words/docword.enron.txt.gz) and understand the file format described at [Bag of Words Data Set](https://archive.ics.uci.edu/ml/datasets/Bag+of+Words).

Step 2. Read the <tt>.gz</tt> file in DIRECTLY.

Step 3. Get the counts for each word using a number transformations and action(s). Note that the triple has a format of "docID wordID count" and you need to get the overall counts for each wordID. This format is different from Exercise 4 of Coursework 1.

Step 4. Set your job to be executed using only one core (thread).

Step 5. Set the number of partitions to be 20, 40, and 80. Time the execution and report the time for each of the three cases.

Step 6. Set your job to be executed using more than one cores (threads) and explain your choice. Repeat Step 5.

Step 7. Briefly describe 3-5 interesting findings.


## Solution Exercise 1

1 core, 1 partition = 5 seconds

1 core, 20 partitions = 16 seconds

1 core, 40 partitions = 15 seconds

1 core, 80 partitions = 15 seconds

7 cores, 20 partitions = 4 seconds

14 cores, 40 partitions = 3 seconds

27 cores, 80 partitions = 4 seconds


> The faster execution time is when every 1 core uses 2-4 partitions (average 3). IE. for 7 cores - 21 partitions.
> By increasing the number of partitions alone, the execution time increases instead of decreasing.
> Also, by increasing the number of cores alone, does not decrease the time. Additionally, its a waste to use a lot of cores and have few number of partitions because you are not utilizing the cores well.

## Exercise 2 [5 marks]

In this exercise, you will perform sentiment analysis using a decision tree. Each row in the file [Sentiment Analysis](./files/sentiment_data.txt) contains a review obtained from Amazon, Yelp or IMDB, and a score, where a score of one indicates a positive comment, and a score of zero indicates a negative comment. Provide a pipeline for feature extraction, and classification using decision trees, and report a table with the different performances that you obtain depending on the parameters used in the pipeline. Which configuration in the pipeline provides the best performance?


## Solution Exercise 2


| MaxDepth | Impurity | MaxBins | ACCURACY |
|----------|----------|---------|----------|
|   10     |  gini    |   10    |   0.641  |
|   10     |  gini    |   20    |   0.626  |
|   10     |  gini    |   30    |   0.630  |
|   10     |  entropy |   10    |   0.651  |
|   10     |  entropy |   20    |   0.652  |
|   10     |  entropy |   30    |   0.658  |
|   15     |  gini    |   10    |   0.658  |
|   15     |  gini    |   20    |   0.667  |
|   15     |  gini    |   30    |   0.656  |
|   15     |  entropy |   10    |   0.634  |
|   15     |  entropy |   20    |   0.641  |
|   15     |  entropy |   30    |   0.664  |
|   20     |  gini    |   10    |   0.664  |
|   20     |  gini    |   20    |   0.655  |
|   20     |  gini    |   30    |   0.666  |
|   20     |  entropy |   10    |   0.671  |
|   20     |  entropy |   20    |   0.656  |
|   20     |  entropy |   30    |   0.660  |


The highest accuracy (0.671) is obtained with MaxDepth = 20, Impurity = Entropy, MaxBins = 10. Therefore, by using these parameters training and testing was altered to see if it affects the accuracy:

| Training| Testing | ACCYRACY |
|---------|---------|----------|
|  0.8    |   0.2   |   0.669  |
|  0.7    |   0.3   |   0.671  |
|  0.6    |   0.4   |   0.670  |
|  0.5    |   0.5   |   0.660  |
|  0.4    |   0.6   |   0.659  |


Therefore, the pipeline configuration that provides the highest accuracy is:

| Training | Testing | MaxDepth | Impurity | MaxBins | ACCURACY |
|----------|---------|----------|----------|---------|----------|
|    0.7   |   0.3   |   20     | Entropy  |   10    |  0.671   |






In [6]:
val sparkVersion = "2.0.1"
val scalaVersion = scala.util.Properties.versionNumberString

sparkVersion: String = "2.0.1"
scalaVersion: String = "2.11.8"

In [7]:
classpath.add(
    "org.apache.spark" %% "spark-yarn" % sparkVersion,
    "org.apache.spark" %% "spark-mllib" % sparkVersion
)

0 new artifact(s)


In [8]:
// Exercise 1:
import org.apache.spark.mllib.linalg._
import org.apache.spark.rdd.RDD
import scala.io.Source
import org.apache.spark.sql.functions._
import org.apache.spark.sql.SparkSession
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.sql.functions.col

object E_1 {
    
    def main(args: Array[String]) {
        
        val sparkSession = SparkSession.builder
            .master("local")
            .appName("E1")
            .getOrCreate()
        
        val sc = sparkSession.sparkContext
        
        import sparkSession.implicits._
        
        
        //read file DIRECTLY:
        val enronDF = sparkSession.sparkContext.textFile("files/docword.enron.txt.gz")
       
       
        val df = enronDF.map(line => line.split(" ")).
                      filter(lines => lines.length == 3). 
                      map(row => (row(0), row(1), row(2))).
                      toDF("docID", "wordID", "count1") 
        //df.show()
        
        val timing = System.nanoTime
        
        val df_partitions = df.repartition(20)
        val get_counts = df_partitions.select($"wordID", $"count1").groupBy("wordID").agg(sum($"count1").as("count")).show()
        
        println((System.nanoTime-timing) / 1e9d)
}
}
E_1.main(Array())

+------+------+
|wordID| count|
+------+------+
| 18130|1989.0|
| 14204|1075.0|
| 26005|  74.0|
| 23843|2179.0|
| 17686|6009.0|
| 23097|  74.0|
| 12394| 183.0|
| 11888| 612.0|
| 24269| 621.0|
| 14369|  23.0|
| 20569| 904.0|
| 21452| 122.0|
| 14157| 281.0|
| 25555|1069.0|
| 14887|1162.0|
| 23459|  84.0|
|  1159| 559.0|
| 25032| 532.0|
|   467| 278.0|
| 18726| 170.0|
+------+------+
only showing top 20 rows

8.390005701


import org.apache.spark.mllib.linalg._
import org.apache.spark.rdd.RDD
import scala.io.Source
import org.apache.spark.sql.functions._
import org.apache.spark.sql.SparkSession
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.sql.functions.col
defined object E_1

In [12]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.DecisionTreeClassificationModel
import org.apache.spark.ml.classification.DecisionTreeClassifier
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}
import org.apache.spark.ml.feature.{RegexTokenizer, Tokenizer}
import org.apache.spark.ml.feature.StopWordsRemover
import org.apache.spark.ml.feature.{CountVectorizer, CountVectorizerModel}
// decision tree imports
import org.apache.spark.mllib.tree.DecisionTree
import org.apache.spark.mllib.tree.model.DecisionTreeModel

// importing CSV data into the expected format
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.linalg.Vectors

import org.apache.spark.sql.Row

import org.apache.spark.sql.functions._


object E_2 {
    
    def main(args: Array[String]) {
        
        val sparkSession = SparkSession.builder
            .master("local")
            .appName("E2")
            .getOrCreate()
        
        val sc = sparkSession.sparkContext
        
        import sparkSession.implicits._



val df = sparkSession.read.format("com.databricks.spark.csv")
    .option("delimiter", "\t")
    .load("files/sentiment_data.txt").toDF

//rename columns:
val col_1 = Seq("features", "label")
               
val df_new = df.toDF(col_1:_*)

        df_new.show()
        

// Index labels, adding metadata to the label column.
// Fit on whole dataset to include all labels in index.
val labelIndexer = new StringIndexer()
  .setInputCol("label")
  .setOutputCol("indexedLabel")
  .fit(df_new)
        
        
val tokenisation = new Tokenizer().setInputCol("features").setOutputCol("tokens")
        
val stopword = new StopWordsRemover().setInputCol("tokens").setOutputCol("stopwordfree")  

   
// fit a CountVectorizerModel from the corpus
val CountVectorizerModel = new CountVectorizer()
  .setInputCol("stopwordfree")
  .setOutputCol("Features")  

        
// Train a DecisionTree model.
val dt = new DecisionTreeClassifier()
  .setLabelCol("indexedLabel")
  .setFeaturesCol("Features")
  .setMaxDepth(20)
  .setImpurity("entropy")
  .setMaxBins(10)
        
// Convert indexed labels back to original labels.
val labelConverter = new IndexToString()
  .setInputCol("prediction")
  .setOutputCol("predictedLabel")
  .setLabels(labelIndexer.labels)

// Split the data into training and test sets (30% held out for testing).
val Array(trainingData, testData) = df_new.randomSplit(Array(0.7, 0.3))

// Pipeline.
val pipeline = new Pipeline()
  .setStages(Array(labelIndexer, tokenisation, stopword, CountVectorizerModel, dt, labelConverter))
        

// Train model. This also runs the indexers.
val model = pipeline.fit(trainingData)

// Make predictions.
val predictions = model.transform(testData)

        

// Select (prediction, true label) and compute test error.
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("indexedLabel")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")
val accuracy = evaluator.evaluate(predictions)
println("Accuracy: "  + (accuracy))

predictions.show()

        
}
}
E_2.main(Array())

+--------------------+-----+
|            features|label|
+--------------------+-----+
|Wow... Loved this...|    1|
|  Crust is not good.|    0|
|Not tasty and the...|    0|
|Stopped by during...|    1|
|The selection on ...|    1|
|Now I am getting ...|    0|
|Honeslty it didn'...|    0|
|The potatoes were...|    0|
|The fries were gr...|    1|
|      A great touch.|    1|
|Service was very ...|    1|
|  Would not go back.|    0|
|The cashier had n...|    0|
|I tried the Cape ...|    1|
|I was disgusted b...|    0|
|I was shocked bec...|    0|
| Highly recommended.|    1|
|Waitress was a li...|    0|
|This place is not...|    0|
|did not like at all.|    0|
+--------------------+-----+
only showing top 20 rows

Accuracy: 0.6446886446886447
+--------------------+-----+------------+--------------------+--------------------+-------------+--------------------+----------+--------------+
|            Features|label|indexedLabel|              tokens|        stopwordfree|rawPrediction|       

import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.DecisionTreeClassificationModel
import org.apache.spark.ml.classification.DecisionTreeClassifier
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}
import org.apache.spark.ml.feature.{RegexTokenizer, Tokenizer}
import org.apache.spark.ml.feature.StopWordsRemover
import org.apache.spark.ml.feature.{CountVectorizer, CountVectorizerModel}
import org.apache.spark.mllib.tree.DecisionTree
import org.apache.spark.mllib.tree.model.DecisionTreeModel
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions._
defined object E_2